# エンコーダ・デコーダによる計算機作成

## 目的
再帰型ニューラルネットワークの構造を理解する

エンコーダ・デコーダモデルの構造を理解する


## エンコーダ・デコーダモデル

リカレントニューラルネットワークは，系列データ内の関連性を内部状態として保持することができます．
この内部状態を利用して，新たな出力ができるようにした構造としてエンコーダ・デコーダがあります．
エンコーダ側に系列データを入力して，中間層では系列データ内の関連性を内部状態を形成します．
デコーダ側には内部状態を与えることで，内部状態を反映した何かしらの結果を出力します．
この応用が，google 翻訳などの機械翻訳です．

<img src="https://drive.google.com/uc?export=view&id=1zFl4Mjo4IRSQWSczJ4PzPkd53YJkb1oM" width = 100%>



## モジュールのインポートとGPUの確認

必要なモジュールをインポートします．
そして，GPUが使用可能かどうかを確認します．

In [ ]:
import random
import numpy as np
from time import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

Use CUDA: True


## このノートブックで行う問題設定

ここでは**文字として**計算式（足し算）を入力して，**文字として**足し算の結果を出力するネットワークを構築します．

具体的には，"123+39"のような足し算式の文字列をLSTMEncoderへと入力し，
"162"のような足し算の結果の文字列をLSTMDecoderから出力させます．

このとき，LSTMには数字の文字や"+"などの記号の文字をひとつづつ入力・出力させます．

## データセットの作成

文字とそれに対応したIDを整理した辞書型オブジェクトを作成します．

`word2id`では文字をキーとしてIDをデータにもつ辞書を，一方`id2word`ではIDをキーとして文字列をデータに持つ辞書を作成します．

作成した辞書を表示します．
`word2id`では，0から9の文字は0~9の数字のキーに対応しており，
各種記号は次のようなIDに対応しています．
* `<pad>`：10
* `+`：11
* 文字列の終わり`<eos>`：12




In [ ]:
word2id = {str(i): i for i in range(10)}
word2id.update({"<pad>": 10, "*": 11, "<eos>": 12})
id2word = {v: k for k, v in word2id.items()}

# 作成した辞書オブジェクトの表示
print(word2id)
print(id2word)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '<pad>': 10, '*': 11, '<eos>': 12}
{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '<pad>', 11: '*', 12: '<eos>'}


### データセットクラスの作成

次に，データセットを用意します．

データは0から9までの数字と加算記号，開始，終了のフラグです．
また，３桁の数字の足し算を行うため，各桁の値を１つずつランダムに生成して連結しています．

$$x + y = z$$







In [ ]:
class CalcDataset(torch.utils.data.Dataset):

    # 計算式の文字列をIDの配列に変換するための関数
    def transform(self, string, seq_len=7):
        tmp = []
        for i, c in enumerate(string):
            try:
                tmp.append(word2id[c])
            except:
                tmp += [word2id["<pad>"]] * (seq_len - i)
                break
        return tmp

    def __init__(self, data_num, train=True):
        super().__init__()

        self.data_num = data_num   # 準備するデータ（計算式）の数
        self.train = train         # 学習，テストのどちらか
        self.data = []             # 入力データ（足し算式の文字列）を格納するリスト
        self.label = []            # 正解（足し算結果の文字列）を格納するリスト

        # data_numの数だけforループを回して足し算式データをランダムに作成
        for _ in range(data_num):
            # 入力データの作成 (x + y) 
            x = int("".join([random.choice(list("0123456789")) for _ in range(random.randint(1, 3))] )) # 0 ~ 999の適当な数字（整数）を生成
            y = int("".join([random.choice(list("0123456789")) for _ in range(random.randint(1, 3))] )) # 0 ~ 999の適当な数字（整数）を生成
            left = ("{:*<7s}".format(str(x) + "*" + str(y))).replace("*", "<pad>")  # x+yの計算式の文字列を作成
            self.data.append(self.transform(left))  # 作成した計算文字列をID配列に変換してリストに格納

            z = x * y  # 足し算の答えを計算
            right = ("{:*<12s}".format(str(z))).replace("*", "<pad>")  # 答えの数値を文字列に変換
            right = self.transform(right, seq_len=9)                  # 文字列 --> IDの配列に変換
            right = [12] + right         # ['EOS', '答えの数字ID', ...]となるように'EOS'を連結（計算上の仕様）
            right[right.index(10)] = 12  # ['EOS', '答えの数字ID', ..., 'EOS', 'PAD', 'PAD']となるようにIDを変更  12 = EOS
            self.label.append(right)     # 作成した答えのID配列を保存
        
        # リスト --> numpy array形式に変換
        self.data = np.asarray(self.data)
        self.label = np.asarray(self.label)

    def __getitem__(self, item):
        d = self.data[item]
        l = self.label[item]
        return d, l

    def __len__(self):
        return self.data.shape[0]

In [ ]:
# 入力データの作成 (x + y) 
x = int("".join([random.choice(list("0123456789")) for _ in range(random.randint(1, 3))] )) # 0 ~ 999の適当な数字（整数）を生成
y = int("".join([random.choice(list("0123456789")) for _ in range(random.randint(1, 3))] )) # 0 ~ 999の適当な数字（整数）を生成
left = ("{:*<7s}".format(str(x) + "*" + str(y))).replace("*", "<pad>")  # x+yの計算式の文字列を作成
print("x:",x)
print("y:",y)
print("left:",left)

x: 33
y: 24
left: 33<pad>24<pad><pad>


In [ ]:
z = x * y  # 足し算の答えを計算
right = ("{:*<12s}".format(str(z))).replace("*", "<pad>")  # 答えの数値を文字列に変換
print("z:",z)
print("right:",right)

z: 792
right: 792<pad><pad><pad><pad><pad><pad><pad><pad><pad>


### 作成したデータの確認

作成した`CalcDataset`のデータを確認します．

適当なデータセットとして`tmp_dataset`を作成します．今回はデータの確認を行うだけのため，データ数（`data_num`）は5と小さい数に指定します．

そして，作成したデータセット内のデータをひとつづつfor文で読み出して，データを確認します．

In [ ]:
tmp_dataset = CalcDataset(data_num=5)

for i in range(len(tmp_dataset)):
    print(tmp_dataset[i])

(array([ 0, 10, 10, 10, 10, 10, 10]), array([12,  0, 12, 10, 10, 10, 10, 10, 10, 10]))
(array([ 5,  2, 10, 10, 10, 10, 10]), array([12,  2,  6,  0, 12, 10, 10, 10, 10, 10]))
(array([ 9,  0, 10, 10, 10, 10, 10]), array([12,  5,  4,  0, 12, 10, 10, 10, 10, 10]))
(array([ 6,  0, 10, 10, 10, 10, 10]), array([12,  3,  7,  2,  0, 12, 10, 10, 10, 10]))
(array([ 8,  3, 10, 10, 10, 10, 10]), array([12,  1,  9,  9,  2, 12, 10, 10, 10, 10]))


## ネットワークモデル（計算機）の定義
ここでは，エンコーダ・デコーダ構造で計算機（足し算）を作ってみます．
このエンコーダ・デコーダ構造のことをSequence-to-Sequence (Seq2Seq) と呼びます．

エンコーダとデコーダの2種類のネットワークを用意します．
エンコーダは，ワードエンベディング (word embedding) という入力されたIDを特徴表現に変換する層とLSTM層から構成されています．
デコーダも同様の構造です．エンコーダ側の中間層の値がstateとして出力され，デコーダ側の中間層に入力されます．

In [ ]:
class Encoder(nn.Module):
    # vocab_size: 扱うIDの数, embedding_dim: embedding層の特徴次元数, hidden_dim: LSTMの隠れ層サイズ
    def __init__(self, vocab_size, embedding_dim, hidden_dim, batch_size=100):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        # padding_idx: padのID (10) を指定（このIDが入力された場合は出力が全て0になる）
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=word2id["<pad>"])
        # batch_first: 入力データの1次元目がミニバッチかどうか（Trueの場合...[mini batch, seqence, feature] となる）
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)                                                                ######

    def forward(self, indices):
        embedding = self.word_embeddings(indices)
        # 配列サイズの確認と適宜サイズ変更（embeddingのサイズが2次元配列の場合には1次元追加して[mini batch, seqence, feature]の形にする）
        if embedding.dim() == 2:
            embedding = torch.unsqueeze(embedding, 1)
        h = torch.zeros(1, self.batch_size, self.hidden_dim, device=device)
        c = torch.zeros(1, self.batch_size, self.hidden_dim, device=device)
        # データをLSTMへ一度に入力し，最後のデータを入れ終わった後の隠れ状態とセル状態（state）を取得
        _, state = self.lstm(embedding, (h, c))
        #_, state = self.lstm(embedding, h)
        return state


class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, batch_size=100):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=word2id["<pad>"])
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.output = nn.Linear(hidden_dim, vocab_size)  # 各IDのスコアが出力されるようにIDの数と同一の出力サイズにする

    def forward(self, index, state):
        embedding = self.word_embeddings(index)
        if embedding.dim() == 2:
            embedding = torch.unsqueeze(embedding, 1)
        lstm_out, state = self.lstm(embedding, state)
        output = self.output(lstm_out)  # lstm_outを全結合層（出力層）へ入力して計算結果の文字（各ID）のスコアを取得
        return output, state

## ネットワークモデルの作成

上で定義したエンコーダとデコーダを作成します．
エンコーダとデコーダは別々のネットワークとして用意し，それぞれの最適化にはAdamを利用します．

In [ ]:
embedding_dim = 16
hidden_dim = 128
vocab_size = len(word2id)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

encoder = Encoder(vocab_size, embedding_dim, hidden_dim, batch_size=100).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_dim, batch_size=100).to(device)

# 正解ラベルにPAD (ID=10) が入力された場合は誤差を計算しない
criterion = nn.CrossEntropyLoss(ignore_index=word2id["<pad>"])

# 最適化手法の設定
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)

###学習
学習を行います．学習データを2万サンプル生成して，データローダに与えます．
学習は200エポック行います．エンコーダの入力は数字または開始・終了・加算記号です．
デコーダの入力は計算結果です．
具体的には，54+37 を行う時，
エンコーダには，まず開始記号を最初に入力し，次に，5, 4, +, 3, 7 を入力します．そして，最後に終了記号を入力します．その時の中間層の情報をhidden_stateとしてエンコーダから受け取ります．
デコーダは，開始記号と中間情報(hidden_state)を最初に入力します，そして，計算結果の9, 1 を入力し，最後に終了記号を入力します．
この時，デコーダは各数字（または記号）の確率をdecoder_outputとして出力します．
decoder_outputは，[バッチサイズ, 1, 各クラス確率]の３次元なので，squeezeによって，[バッチサイズ,  各クラス確率] に次元削減します．
そして，クロスエントロピー誤差関数によって，ロスを求めます．
これを正解の長さ(=5)分繰り返し行い，ロスを累積します．
その後，誤差逆伝播，デコーダ，エンコーダの更新を行います．


In [ ]:
batch_size=100
epoch_num = 50

train_data = CalcDataset(data_num = 20000)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

encoder.train()
decoder.train()

start = time()
for epoch in range(1, epoch_num+1):
    for data, label in train_loader:
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        if use_cuda:
            data = data.cuda()
            label = label.cuda()

        encoder_hidden = encoder(data)
        source = label[:, :-1]  # 学習時にデコーダに入力するデータを抽出
        target = label[:, 1:]   # 正解ラベルを抽出
        decoder_hidden = encoder_hidden  # エンコーダの隠れ・セル状態をデコーダへ渡すためにコピー

        loss = 0
        for i in range(source.size(1)):
            decoder_output, decoder_hidden = decoder(source[:, i], decoder_hidden)
            decoder_output = torch.squeeze(decoder_output)
            loss += criterion(decoder_output, target[:, i])

        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

    elapsed_time = time() - start
    if epoch % 10 == 0:
        print("epoch: {}, mean loss: {}, elapsed_time: {}".format(epoch, loss.item(), elapsed_time))

# 学習が一通り終了した時点で，ネットワークモデルのパラメータを保存
model_name = "seq2seq_calculator_v{}.pt".format(epoch)
torch.save({
    'encoder_model': encoder.state_dict(),
    'decoder_model': decoder.state_dict(),
}, model_name)

epoch: 10, mean loss: nan, elapsed_time: 20.551557779312134


KeyboardInterrupt: ignored

## 評価

次に，学習したモデルを評価をします．

テストデータを50サンプル生成して，データローダに与えます．

ここで，学習時はエンコーダとデコーダのバッチサイズを100としていました．
テスト時は１つずつ行いたいので，エンコーダとデコーダを新たに生成し，学習したパラメータをロードします．

エンコーダ側に計算したい数字（または記号）を入力して中間情報stateを得ます．
デコーダ側に，中間情報stateと開始記号<eos>を入力します．
デコーダ側の出力は数字または記号(token)と中間情報です．
これらを繰り返しデコーダに入力します．<eos>が出力されたら繰り返しは終了です．

出力されたtokenを追加したリストrightを計算結果とします．
計算する式(left)を作成した後，evalでその計算結果が正しいかどうかを判定します．



### GPUが使用できず，学習ができなかった場合

学習済みモデルを用意していますので，下記のコマンドを実行してファイルをダウンロードしてください．

In [ ]:
!wget -qq http://www.mprg.cs.chubu.ac.jp/~hirakawa/share/tutorial_data/seq2seq_calculator_v200.pt.zip
!unzip seq2seq_calculator_v200.pt.zip

Archive:  seq2seq_calculator_v200.pt.zip
replace seq2seq_calculator_v200.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
batch_size = 1
test_data = CalcDataset(data_num = 50)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

encoder = Encoder(vocab_size, embedding_dim, hidden_dim, batch_size=1).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_dim, batch_size=1).to(device)

model_name = "seq2seq_calculator_v{}.pt".format(epoch)
checkpoint = torch.load(model_name)
encoder.load_state_dict(checkpoint["encoder_model"])
decoder.load_state_dict(checkpoint["decoder_model"])

encoder.eval()
decoder.eval()

accuracy = 0
        
# 評価の実行   
with torch.no_grad():
    for data, label in test_loader:
        if use_cuda:
            data = data.cuda()

        # encoderの計算
        state = encoder(data)

        # decoderの計算
        right = []
        token = "<eos>"
        for _ in range(7):
            index = word2id[token]  # decoderに入力するIDを決定（最初はEOS, 次から前の時刻のdecoderの出力）
            input_tensor = torch.tensor([index], device=device)  # IDをtorchの配列形式に変換
            output, state = decoder(input_tensor, state)         # IDを入力
            prob = F.softmax(torch.squeeze(output), dim=0)       # softmaxを計算
            index = torch.argmax(prob.cpu().detach()).item()     # 出力の中で最もスコアの高いIDを決定
            token = id2word[index]                               # そのIDを文字に変換
            if token == "<eos>":  # 予測結果がEOSなら終了
                break
            right.append(token)                                  # 文字に変換した予測結果をリストに格納
        right = "".join(right)
        
        # 計算式（左辺）のID配列 --> 文字列に変換（表示用）
        x = list(data[0].to('cpu').detach().numpy())
        try:
            padded_idx_x = x.index(word2id["<pad>"])
        except ValueError:
            padded_idx_x = len(x)
        left = "".join(map(lambda c: str(id2word[c]), x[:padded_idx_x]))

        # 正解判定
        try:
          right_int = int(right)          # 予測結果の文字列を数値に変換
          flag = eval(left) == right_int  # 正しければTrueを保存
        except:
          flag = False

        print("{:>7s} = {:>4s}".format(left, right), flag)  # 計算結果の表示

        if flag:
            accuracy += 1   # 正解した場合はカウントする

print("Accuracy: {:.2f}".format(accuracy / len(test_loader)))

FileNotFoundError: ignored

## 課題

### 1. 他のリカレントニューラルネットワークを使って精度比較をしてみましょう．

**ヒント**

その他のネットワークとしては`nn.RNN`や`nn.GRU`があります．

また，RNNやGRUにはセル状態がないため，変数`c`を使用しないように注意
```
self.lstm(embedding, (h, c)) --> self.lstm(embedding, h)
```


### 2. 足し算だけでなく，色々な四則演算を実装しましょう．
**こちらは時間があれば取り組んでみましょう**